In [1]:
import os
import random

In [34]:
def count_direct_subfolders(root_folder):
    return sum(1 for item in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, item)))

def count_subfolders(folder_path):
    total_folders = sum(len(dirs) for root, dirs, _ in os.walk(folder_path))
    direct_subfolders_count = count_direct_subfolders(folder_path)
    return total_folders - direct_subfolders_count

def select_folders(data_folder_1, artificialdata_folder, percentage):
    # Get list of folders in data_folder_1
    data_folders_1 = [f for f in os.listdir(data_folder_1) if os.path.isdir(os.path.join(data_folder_1, f))]
    
    # Get list of folders in data_folder_2 within artificialdata_folder
    data_folder_2_path = os.path.join(artificialdata_folder, 'experiment_3', 'data')
    data_folders_2 = [os.path.join(root, dir) for root, dirs, files in os.walk(data_folder_2_path) for dir in dirs]
    
    # Calculate the total number of folders within data_folder_1
    total_folders_in_data_folder_1 = count_subfolders(data_folder_1)
    
    # Calculate the total number of folders within data_folder_2
    total_folders_in_data_folder_2 = count_subfolders(data_folder_2_path)
    
    # Calculate the maximum number of folders that can be selected
    max_folders_to_select = min(total_folders_in_data_folder_2, int(total_folders_in_data_folder_2 * (percentage / 100)))
    
    # Randomly select folders from data_folder_2
    num_folders_to_select = min(max_folders_to_select, len(data_folders_2))
    selected_folders = random.sample(data_folders_2, num_folders_to_select)
    
    # Print the total number of subfolders in data_folder_1
    print(f"Total number of subfolders in {data_folder_1}: {total_folders_in_data_folder_1}")
    
    return selected_folders

In [36]:
data_folder_1 = '../../data'
artificialdata_folder = '../../artificialdata'
percentage = 20  

selected_folders = select_folders(data_folder_1, artificialdata_folder, percentage)
print("Selected folders from artificialdata folder:")
for folder in selected_folders:
    print(folder)


Total number of subfolders in ../../data: 25
Selected folders from artificialdata folder:
../../artificialdata/experiment_3/data/PML_RARA/patient_3
../../artificialdata/experiment_3/data/RUNX1_RUNX1T1/patient_4
../../artificialdata/experiment_3/data/CBFB_MYH11/patient_5
../../artificialdata/experiment_3/data/control/patient_1
../../artificialdata/experiment_3/data/NPM1/patient_5
